In [62]:
import pandas as pd
import os
import glob
import numpy as np
import re

In [63]:
## For each text file in folder:
## 1) Create an excel extract
## 2) compare all matrix values and return:
##    a) occurrences of values (number of times a correlation was present)
##    b) Z-Score from correlation coefficient
##    c) Updated Correlation coefficient from an averaged z-score
# getting csv files from the folder MyProject
path = os.getcwd()

# read all the files with extension .csv
filenames = glob.glob(path + "\*.txt")
newpath = path+'/Extract'
if not os.path.exists(newpath):
   os.makedirs(newpath)

d_list = []
z_list = []
# for loop to iterate all txt files
# save copy for edit
for file in filenames:
   # reading csv files
   d = pd.read_csv(file, sep=';')
   
   #drop duplicated columns: columns with period present
   d.drop([col for col in d.columns if '.' in col],axis=1, inplace=True)
   #drop duplicated rows
   d = d.drop_duplicates(subset=['Matrix'])

   file_name = file.rsplit(' ', 1)[1]
   file_name = file_name.rsplit('.', 1)[0]   
   #d.to_excel(newpath+"\\"+file_name + ".xlsx", index = False)  
   #z-score
   d_indexed = d.set_index('Matrix')
   #drop duplicated columns by identifying where columns have a period
   
   
   ##(1/2)log( (1+r)/(1-r))
   numeric_cols = [col for col in d_indexed if d_indexed[col].dtype.kind != 'O']
   d_z_score = (1+d_indexed[numeric_cols]) / (1-d_indexed[numeric_cols])
   d_z_score = np.log10(d_z_score) / 2
   d_z_score.reset_index(drop=False, inplace=True)

   #Export
   # create a excel writer object
   with pd.ExcelWriter(newpath+"\\"+file_name + ".xlsx") as writer:
      d.to_excel(writer, sheet_name="Extract", index=False)
      d_z_score.to_excel(writer, sheet_name="Z Scores", index=False)

   d_list.append(d)
   z_list.append(d_z_score)

df_final=pd.concat(d_list)
z_score=pd.concat(z_list)

#average Z_score
average_z_score = z_score.groupby('Matrix').mean()
#count of occurences
n_matrix = df_final.groupby('Matrix').count()

#export 
#file name
updated_file_name = file_name.split('_', 1)[-1]
updated_file_name = updated_file_name.split("_")[0]
  
nextpath = path+'/Calculated'
if not os.path.exists(nextpath):
   os.makedirs(nextpath)
# create a excel writer object
with pd.ExcelWriter(nextpath+"\\"+updated_file_name + ".xlsx") as writer:
   average_z_score.to_excel(writer, sheet_name="Average Z", index=True)
   n_matrix.to_excel(writer, sheet_name="Samples", index=True)

In [64]:
d_indexed

,(R) Hippocampal formation,(L) Hippocampal formation,(R) Field CA1,(L) Field CA1,(R) Field CA2,(L) Field CA2,(R) Field CA3,(L) Field CA3,(R) Dentate gyrus,(L) Dentate gyrus,...,(L) Lateral amygdalar nucleus,(L) Basolateral amygdalar nucleus,(R) Medial amygdalar nucleus,(L) Medial amygdalar nucleus,(R) Central amygdalar nucleus,(L) Central amygdalar nucleus,(R) Temporal association areas,(L) Temporal association areas,(R) Isocortex,(L) Isocortex
Matrix,,,,,,,,,,,,,,,,,,,,,
(R) Hippocampal formation,1.000000,0.730102,0.710741,0.526268,0.157807,0.140250,0.504705,0.489510,0.672197,0.654377,...,0.146691,-0.024968,-0.074113,0.019608,0.090126,-0.018501,0.245949,0.449840,0.710386,0.771829
(L) Hippocampal formation,0.730102,1.000000,0.563440,0.736573,0.074036,0.171063,0.293861,0.670551,0.479014,0.862807,...,0.128146,-0.113235,-0.102960,-0.034548,-0.029546,-0.055522,0.191506,0.508765,0.591806,0.738768
(R) Field CA1,0.710741,0.563440,1.000000,0.340364,0.154797,0.193823,0.347705,0.395128,0.534283,0.537499,...,0.150818,-0.079450,-0.056571,0.049177,-0.036933,-0.006255,0.118813,0.376078,0.555134,0.598818
(L) Field CA1,0.526268,0.736573,0.340364,1.000000,-0.009305,0.162693,0.233534,0.454213,0.338716,0.487414,...,-0.006828,-0.066908,-0.010782,-0.026209,-0.080457,-0.163914,0.083221,0.383558,0.435190,0.538857
(R) Field CA2,0.157807,0.074036,0.154797,-0.009305,1.000000,0.002157,0.100253,-0.033692,0.028493,0.093490,...,-0.068340,-0.001983,-0.069467,-0.005413,-0.022420,0.058379,0.029300,0.045348,0.006748,0.010503
(L) Field CA2,0.140250,0.171063,0.193823,0.162693,0.002157,1.000000,0.153519,0.218662,0.166474,0.017042,...,-0.067201,0.042869,0.065060,-0.014625,-0.036502,-0.051195,0.016963,0.153978,0.215225,0.187962
(R) Field CA3,0.504705,0.293861,0.347705,0.233534,0.100253,0.153519,1.000000,0.212089,0.335509,0.309855,...,0.124072,0.065673,0.042430,0.026064,0.026899,-0.070998,0.152877,0.179858,0.383956,0.382053
(L) Field CA3,0.489510,0.670551,0.395128,0.454213,-0.033692,0.218662,0.212089,1.000000,0.369078,0.417598,...,0.085685,-0.121488,-0.060573,0.038879,0.011239,-0.116744,0.172662,0.273030,0.414205,0.513498
(R) Dentate gyrus,0.672197,0.479014,0.534283,0.338716,0.028493,0.166474,0.335509,0.369078,1.000000,0.477939,...,0.112191,-0.059506,-0.021782,0.022878,0.108249,-0.057215,0.185896,0.295834,0.678591,0.632723
